In [ ]:
# iniciando


# Función imputación de outlier
# ------

def imputar_valores_extremos(df, variable, metodo='media'):
    """
    Imputa valores extremos en una variable de un DataFrame utilizando la media o la mediana.

    Parámetros:
    df (DataFrame): El DataFrame que contiene la variable a imputar.
    variable (str): El nombre de la variable que deseas imputar.
    metodo (str): La forma de imputación ('media' o 'mediana'). Por defecto es 'media'.

    Retorna:
    DataFrame: El DataFrame con la variable imputada.
    """
    if metodo not in ['media', 'mediana']:
        raise ValueError("El método debe ser 'media' o 'mediana'")

    # Calcular la media o la mediana
    if metodo == 'media':
        valor_imputacion = df[variable].mean()
    else:
        valor_imputacion = df[variable].median()

    # Identificar valores extremos (usando una regla de 3 veces la desviación estándar)
    limite_inferior = df[variable].mean() - 3 * df[variable].std()
    limite_superior = df[variable].mean() + 3 * df[variable].std()

    # Imputar valores extremos
    df[variable] = np.where(
        (df[variable] < limite_inferior) | (df[variable] > limite_superior),
        valor_imputacion,
        df[variable]
    )

    return df


# Función imputación perdidos
# ------

def imputar_valores(df, variable, metodo='media', valor_especifico=None):
    """
    Imputa valores perdidos en una columna de un DataFrame según el método especificado.

    Parámetros:
    df (pd.DataFrame): El DataFrame en el que se imputarán los valores.
    variable (str): El nombre de la columna a imputar.
    metodo (str): El método de imputación ('media', 'mediana', 'moda', 'valor_especifico').
    valor_especifico: El valor específico a usar para la imputación (relevante solo si 'metodo' es 'valor_especifico').

    Retorna:
    pd.DataFrame: El DataFrame con la variable imputada.
    """

    if metodo == 'media':
        imputacion = df[variable].mean()
    elif metodo == 'mediana':
        imputacion = df[variable].median()
    elif metodo == 'moda':
        imputacion = df[variable].mode()[0]
    elif metodo == 'valor_especifico':
        if valor_especifico is None:
            raise ValueError("Debe proporcionar un valor específico para la imputación.")
        imputacion = valor_especifico
    else:
        raise ValueError("Método de imputación no reconocido. Use 'media', 'mediana', 'moda' o 'valor_especifico'.")

    df[variable].fillna(imputacion, inplace=True)
    return df


# Funcion graficadora confusion_marix
# ---
def confusion_matrix_graph(cm):
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
              xticklabels=['No', 'Yes'],
              yticklabels=['No', 'Yes'])
  plt.title('Matriz de Confusión')
  plt.xlabel('Predicción')
  plt.ylabel('Realidad')
  plt.show()


# Funcion ROC curve
# ---
def roc_curve_graph(y,prob):
  # Obtener las probabilidades de la clase positiva
  y_prob = prob[:, 1]  # Probabilidades de la clase 1

  # Calcular la curva ROC
  fpr, tpr, thresholds = roc_curve(y,  y_prob)

  # Calcular el área bajo la curva (AUC)
  roc_auc = auc(fpr, tpr)

  # Graficar la curva ROC
  plt.figure(figsize=(8, 6))
  plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
  plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Línea de referencia
  plt.title('Curva ROC')
  plt.xlabel('Tasa de Falsos Positivos (FPR)')
  plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
  plt.legend(loc='lower right')
  plt.grid()
  plt.show()



In [ ]:

# Scripts del Proyecto

# Script 1: Preparacion de datos para el entrenamiento
#---------------------------------------------------------

# carga de datos
#Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from collections import Counter


# Leemos la tabla de entrenamiento
data = pd.read_csv("..\data\Data_Customer_Churn.csv",sep = ';')

# 1 Limpieza de datos

## Separando las variables segun su tipo para un correcta lectura
# Lista de variables numéricas
numeric_vars = data.select_dtypes(include=['number']).columns.tolist()
# Lista de variables categóricas
categorical_vars = data.select_dtypes(include=['object', 'category']).columns.tolist()

# Actualizamos variables numericas
data["SeniorCitizen"] = data["SeniorCitizen"].astype("str")
numeric_vars.remove('SeniorCitizen')
categorical_vars.append('SeniorCitizen')
numeric_vars, categorical_vars
# Analizando variables categóricas
# Iterar sobre las columnas del DataFrame
for column in data.columns:
  if data[column].dtype == 'object' or data[column].dtype.name == 'category':
    print(f"Resumen de porcentajes para la variable '{column}':\n")
    print(data[column].value_counts(normalize=True) * 100)
    print("\n" + "-"*50 + "\n")

# Actualizamos variables categoricas
data["TotalCharges"] = data["TotalCharges"].replace(" ",np.nan)
data["TotalCharges"] = data["TotalCharges"].astype("float")
categorical_vars.remove('TotalCharges')
numeric_vars.append('TotalCharges')

# Presencia de valores perdidos
for column in data.columns:
    missing_percentage = data[column].isnull().mean() * 100
    print(f'{column}: {missing_percentage:.2f}%')

# Tenemos valores en nuestra tabla que son espacios en blanco no necesariamente son valores nulos (TIP OPCIONAL)
data = imputar_valores(data,'TotalCharges',metodo='mediana')

for column in data.columns:
    missing_percentage = data[column].isnull().mean() * 100
    print(f'{column}: {missing_percentage:.2f}%')




# 2 Preprocesamiento de datos

# Retirando la variable target de la lista de vaiables categoricas
categorical_vars.remove('Churn')
categorical_vars.remove('customerID')
# Guardar todas las variables categoricas en un solo lugar
cat_cols = data[categorical_vars]
num_cols = data[numeric_vars]
# Generar variables para las dos columnas que omiti de mi mapeo de variables cualitativas y cuantitativas
id_customer = data["customerID"]
label = data["Churn"]

# Transformacion de variables categoricas a numericas
# Label encoding (Target Encoding) : Cambiar en la misma columna el valor categorico a numerico
# La variable target categorica solo puede ser transformada con el target encoding
label = label.apply(lambda x: 1 if x == "Yes" else 0) # Yes - 1, No -0


# Analizando importancia de variables en un modelo simple

from sklearn.tree import DecisionTreeClassifier
# Separar las variables de entrada y la target
X = data[['tenure','TotalCharges']]
y = data['Churn']
# Entrenar un árbol de decisión
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X, y)
# Obtener la importancia de las variables
importances = tree.feature_importances_
print(f'Importancia de tenure: {importances[0]}')
print(f'Importancia de TotalCharges: {importances[1]}')

# retirando la variable que menos aporta
numeric_vars.remove('tenure')
numeric_vars
del num_cols['tenure']

#transformamos las variables categóricas a numéricas
cat_cols = pd.get_dummies(data = cat_cols)

# Creamos una nueva data donde concatenamos la data original con la data que contiene las nuevas variables de one-hot-encoding:
# que será la data final para aplicar los modelos
df_complete = pd.concat([num_cols, cat_cols, label], axis=1)
df_complete.to_csv("../data/processed/churn_train.csv")

In [ ]:

# Script 2: Código de Entrenamiento
#---------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score,roc_curve # ROC a diferencia del accuracy te da un valor justo de precision para datos desbalanceados
# Particionado de datos
# Al dataset o set de entrenamiento le retiramos la variable dependiente o target
X = df_complete.drop("Churn",axis=1) # covariables
Y = df_complete['Churn'] # target

from sklearn.model_selection import train_test_split
X_train_res, X_test, y_train_res, y_test = train_test_split(X, Y, test_size = 0.3, random_state=10)


pip install imblearn
from imblearn.over_sampling import SMOTE
# Aplicar SMOTE al conjunto de entrenamiento
# Instanciar SMOTE
smote = SMOTE(sampling_strategy=0.6, random_state=42)   # Instanciamos el SMOTE
# Aplicar SMOTE
X_train_smote, y_train_smote = smote.fit_resample(X_train_res, y_train_res)
# Seleccionamos el tipo de remuestreo que mejor nos beneficie
X_train = X_train_smote
y_train = y_train_smote

# Entrenamos el modelo con toda la muestra
LG = LogisticRegression()
LG.fit(X_train, y_train)

y_train_predict_lgr =  LG.predict(X_train)
y_test_predict_lgr =  LG.predict(X_test)

y_train_predict_proba_lgr =  LG.predict_proba(X_train)
y_test_predict_proba_lgr =  LG.predict_proba(X_test)

# Calcular la matriz de confusión
cm_lgr = confusion_matrix(y_train, y_train_predict_lgr)
cm_lgr
confusion_matrix_graph(cm_lgr)
# las métricas
from sklearn import metrics
print(metrics.classification_report(y_train,y_train_predict_lgr))
from sklearn import metrics
print(metrics.classification_report(y_test,y_test_predict_lgr))
roc_curve_graph(y_train,y_train_predict_proba_lgr)

# Guardamos el modelo entrenado para usarlo en produccion
import pickle
filename = '..\models\modelo_LG.sav'
pickle.dump(LG, open(filename, 'wb'))


In [ ]:

#Script 3: Preparación de Datos de Validación
#------------------------------------------------
# carga de datos
#Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from collections import Counter


# Leemos la tabla de entrenamiento
data = pd.read_csv("..\data\Data_Customer_Churn_new.csv",sep = ';')

# 1 Limpieza de datos

## Separando las variables segun su tipo para un correcta lectura
# Lista de variables numéricas
numeric_vars = data.select_dtypes(include=['number']).columns.tolist()
# Lista de variables categóricas
categorical_vars = data.select_dtypes(include=['object', 'category']).columns.tolist()

# Actualizamos variables numericas
data["SeniorCitizen"] = data["SeniorCitizen"].astype("str")
numeric_vars.remove('SeniorCitizen')
categorical_vars.append('SeniorCitizen')
numeric_vars, categorical_vars
# Analizando variables categóricas
# Iterar sobre las columnas del DataFrame
for column in data.columns:
  if data[column].dtype == 'object' or data[column].dtype.name == 'category':
    print(f"Resumen de porcentajes para la variable '{column}':\n")
    print(data[column].value_counts(normalize=True) * 100)
    print("\n" + "-"*50 + "\n")

# Actualizamos variables categoricas
data["TotalCharges"] = data["TotalCharges"].replace(" ",np.nan)
data["TotalCharges"] = data["TotalCharges"].astype("float")
categorical_vars.remove('TotalCharges')
numeric_vars.append('TotalCharges')

# Presencia de valores perdidos
for column in data.columns:
    missing_percentage = data[column].isnull().mean() * 100
    print(f'{column}: {missing_percentage:.2f}%')

# Tenemos valores en nuestra tabla que son espacios en blanco no necesariamente son valores nulos (TIP OPCIONAL)
data = imputar_valores(data,'TotalCharges',metodo='mediana')

for column in data.columns:
    missing_percentage = data[column].isnull().mean() * 100
    print(f'{column}: {missing_percentage:.2f}%')




# 2 Preprocesamiento de datos

# Retirando la variable target de la lista de vaiables categoricas
categorical_vars.remove('Churn')
categorical_vars.remove('customerID')
# Guardar todas las variables categoricas en un solo lugar
cat_cols = data[categorical_vars]
num_cols = data[numeric_vars]
# Generar variables para las dos columnas que omiti de mi mapeo de variables cualitativas y cuantitativas
id_customer = data["customerID"]
label = data["Churn"]

# Transformacion de variables categoricas a numericas
# Label encoding (Target Encoding) : Cambiar en la misma columna el valor categorico a numerico
# La variable target categorica solo puede ser transformada con el target encoding
label = label.apply(lambda x: 1 if x == "Yes" else 0) # Yes - 1, No -0

# Analizando importancia de variables en un modelo simple

from sklearn.tree import DecisionTreeClassifier
# Separar las variables de entrada y la target
X = data[['tenure','TotalCharges']]
y = data['Churn']
# Entrenar un árbol de decisión
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X, y)
# Obtener la importancia de las variables
importances = tree.feature_importances_
print(f'Importancia de tenure: {importances[0]}')
print(f'Importancia de TotalCharges: {importances[1]}')

# retirando la variable que menos aporta
numeric_vars.remove('tenure')
numeric_vars
del num_cols['tenure']

#transformamos las variables categóricas a numéricas
cat_cols = pd.get_dummies(data = cat_cols)

# Creamos una nueva data donde concatenamos la data original con la data que contiene las nuevas variables de one-hot-encoding:
# que será la data final para aplicar los modelos
df_complete = pd.concat([num_cols, cat_cols, label], axis=1)
df_complete.to_csv("../data/processed/churn_val.csv")

In [ ]:

# Script 4: Código de Validación
#--------------------------------
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import pickle
import matplotlib.pyplot as plt

# Cargar la tabla transformada
df = pd.read_csv("../data/processed/churn_val.csv")
X_test = df.drop(['Churn'],axis=1)
y_test = df[['Churn']]
# Leemos el modelo entrenado!
filename = '..\models\modelo_LG.sav'
pickle.dump(LG, open(filename, 'wb'))

# Predecimos sobre el set de datos de implementacion con el modelo entrenado
y_pred_test=model.predict(df.drop(['Churn'],axis=1)) 
## Metricas de validación
def calc_metrics(y_test,y_pred_test):
    cm_test = confusion_matrix(y_test,y_pred_test)
    print("Matriz de confusion: ")
    print(cm_test)
    accuracy_test=accuracy_score(y_test,y_pred_test)
    print("Accuracy: ", accuracy_test)
    precision_test=precision_score(y_test,y_pred_test)
    print("Precision: ", precision_test)
    recall_test=recall_score(y_test,y_pred_test)
    print("Recall: ", recall_test)
def save_plot(title):
    plt.title(title)
    fig = plt.gcf()
    filename = title.replace(" ", "_").lower()
    fig.savefig('{}'.format(filename), dpi=500)
    plt.clf()
plot_confusion_matrix(model, X_test, y_test)
save_plot('Confusion Matrix')
plot_roc_curve(model, X_test, y_test)
save_plot('ROC Curve')

